### Identify events which appear twice and delete them

In [2]:
import os
import hashlib
import pandas as pd
import numpy as np

In [28]:
def copy_files(em, gauge_num, option_num):
    # path to source directory
    src_dir = f"/nfs/a319/gy17m2a/PhD/ProcessedData/IndependentEvents/{em}/Gauge{gauge_num}/Option{option_num}/" 
    # path to destination directory
    dest_dir = f"/nfs/a319/gy17m2a/PhD/ProcessedData/IndependentEvents/{em}/Gauge{gauge_num}/Option{option_num}/EventSet/" 

    # getting all the files in the source directory
    files = os.listdir(src_dir)
    
    if not os.path.isdir(dest_dir):
        shutil.copytree(src_dir, dest_dir)

def hash_dataframe(filepath, columns_to_exclude=None):
    """
    Generate a hash for a DataFrame loaded from a CSV file,
    excluding a specified column if provided.
    """
    # Load the CSV file into a DataFrame
    df = pd.read_csv(filepath)
    
    # Drop the specified column if it exists
    for column_to_exclude in columns_to_exclude:
        if column_to_exclude and column_to_exclude in df.columns:
            df = df.drop(columns=[column_to_exclude])
    
    # Convert DataFrame to a binary format using 'to_records' which includes data types in the hash
    data = df.to_records(index=False)
    return hashlib.sha256(data.tobytes()).hexdigest()

def find_and_remove_duplicates(directory, column_to_exclude=None):
    """
    Find and remove duplicate CSV files in a given directory,
    excluding comparison on a specified column.
    """
    hashes = {}
    files = [f for f in os.listdir(directory) if f.endswith('.csv')]
    files = np.sort(files)
    
    for filename in files:
        filepath = os.path.join(directory, filename)
        file_hash = hash_dataframe(filepath, column_to_exclude)
        
        if file_hash in hashes:
            # If hash is already in the dictionary, delete this file
            os.remove(filepath)
            print(f"Deleted duplicate file: {filepath}")
        else:
            # Otherwise, add the hash and file path to the dictionary
            hashes[file_hash] = filepath

# Usage
for gauge_num in range(40,47):
    print(gauge_num)
    em = 'bc005'
    copy_files(em, gauge_num, option_num =1)
    copy_files(em, gauge_num, option_num =2)
    directory1 = f"/nfs/a319/gy17m2a/PhD/ProcessedData/IndependentEvents/{em}/Gauge{gauge_num}/Option1/EventSet/" 
    directory2 = f"/nfs/a319/gy17m2a/PhD/ProcessedData/IndependentEvents/{em}/Gauge{gauge_num}/Option2/EventSet/" 
    column_to_exclude = 'Rolling_Sum'  # Change this to the column you want to exclude
    find_and_remove_duplicates(directory1, column_to_exclude)
    find_and_remove_duplicates(directory2, column_to_exclude)

40
41
42
43
44
45
46


In [3]:
def is_sequential(numbers):
    return all(numbers[i] - numbers[i - 1] == 1 for i in range(1, len(numbers)))

# Loop through gauges
for gauge_num in [37]:
    
    for em in ['bc005']:
        directory = f"../../ProcessedData/IndependentEvents/{em}/Gauge{gauge_num}/Option2"
        files = [f for f in os.listdir(directory) if f.endswith('.csv')]
        files = np.sort(files)
        for file in files:
            test = pd.read_csv(directory+ '/'+ file)
            if test['Unnamed: 0'].is_monotonic_increasing ==False:
                print(f"monotonic: {file}")
            if not is_sequential(test['Unnamed: 0']):
                print(f"not sequential {file}")
